<a href="https://colab.research.google.com/github/clemsadand/AirPollution/blob/main/Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/clemsadand/AirPollution.git

Cloning into 'AirPollution'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 74 (delta 38), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 333.38 KiB | 748.00 KiB/s, done.
Resolving deltas: 100% (38/38), done.


In [2]:
!sudo apt-get update -q
!sudo apt-get install -y libglu1-mesa -q

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,964 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchp

In [3]:
!pip install -r AirPollution/requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.8/39.8 MB 10.7 MB/s eta 0:00:00
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=8171d050c5e389596260066eaf9ccea51525bbf9bbadef43f9356b0cb592d333
  Stored in directory: /root/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successfully built pyDOE


In [4]:
# !python AirPollution/pinn2.py

In [5]:
# !python AirPollution/crbe_fem.py

| Hyperparameter | Values |
|---|---|
| Learning rate | 10⁻⁴, 5×10⁻⁴, 10⁻³, 5×10⁻³ |
| Network depth | 3, 4, 5, 6 hidden layers |
| Network width | 32, 64, 128 neurons per layer |
| Loss weights (λ_PDE) | 0.1, 1.0, 10.0 |
| Loss weights (λ_IC, λ_BC) | 1.0, 10.0, 100.0 |
| Activation functions | tanh, sine, swish |

In [1]:
learning_rates = [1e-4, 5e-4, 1e-3, 5e-3]
network_depths = [3, 4, 5, 6]
network_widths = [16, 32, 64]
loss_weights_pde = [0.1, 1.0, 10.0]
loss_weights_ic_bc = [1.0, 10.0, 100.0]
activation_functions = ["tanh", "sine", "swish"]

In [5]:
n_steps = 32
mesh_size = 64

mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
mesh = meshio.read(mesh_file)
domains = Domains(Lx=Lx, Ly=Ly, T=T)
mesh_data = crbe_fem.MeshData(mesh, domains, nt=n_steps)

# ========= Hyperparameter
epochs = 20

n_ic = round(0.2 * mesh_data.number_of_segments)
n_bc = n_ic
n_col = mesh_data.number_of_segments - n_ic - n_bc
batch_sizes = {'pde': n_col, 'ic': n_ic, 'bc': n_ic}

print(batch_sizes)


{'pde': 8398, 'ic': 2800, 'bc': 2800}


In [ ]:
import meshio
import AirPollution.pinn2 as pinn2
import AirPollution.crbe_fem as crbe_fem
from AirPollution.crbe_fem import create_mesh, Domains, Models, MeshData
import numpy as np
# Set random seed for reproducibility
np.random.seed(1234)


# ========== Model params
domain_size = 20.0
domain = [-domain_size, domain_size] * 2

T = 10.0  # End time
time_range = [0, T]

Lx = Ly = domain_size  # Half-size of the domain
D = 0.1  # Diffusion coefficient (small value leads to advection-dominated flow)
vx, vy = 1.0, 0.5  # Velocity field
v = (vx, vy)
sigma = 0.1

model_params = crbe_fem.Models(vx=vx, vy=vy, D=D, sigma=sigma)

# ========== Mesh Data
n_steps = 32
mesh_size = 16

mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
mesh = meshio.read(mesh_file)
domains = Domains(Lx=Lx, Ly=Ly, T=T)
mesh_data = crbe_fem.MeshData(mesh, domains, nt=n_steps)

# ========= Hyperparameter
epochs = 20

n_ic = round(0.2 * mesh_data.number_of_segments)
n_bc = n_ic
n_col = mesh_data.number_of_segments - n_ic - n_bc
batch_sizes = {'pde': n_col, 'ic': n_ic, 'bc': n_ic}


# ========= Results
pinn_results = []



# ========== Grid search

for lr in learning_rates:
    for depth in network_depths:
        for width in network_widths:
            for lambda_pde in loss_weights_pde:
                for lambda_ic_bc in loss_weights_ic_bc:
                    for activation in activation_functions:
                        print(f"Training with lr={lr}, depth={depth}, width={width}, lambda_pde={lambda_pde}, lambda_ic_bc={lambda_ic_bc}, activation={activation}")
                        layers = [3] + [width] * depth + [1]
                        lambda_weights = {'pde': lambda_pde, 'ic': lambda_ic_bc, 'bc': lambda_ic_bc}
                        model = pinn2.PINN(layers, D, v, activation)

                        history = pinn2.train_pinn(model, domain, time_range, batch_sizes, lr, epochs, lambda_weights)
                        x_eval_flat = mesh_data.midpoints[:,0].reshape(-1, 1)
                        y_eval_flat = mesh_data.midpoints[:,0].reshape(-1, 1)
                        t_eval = np.ones_like(x_eval_flat) * T

                        rel_l2_error, max_error, u_pred, u_exact = pinn2.compute_error(
                                    model, t_eval, x_eval_flat, y_eval_flat)

                        pinn_results.append({
                            'lr': lr,
                            'depth': depth,
                            'width': width,
                            'lambda_pde': lambda_pde,
                            'lambda_ic_bc': lambda_ic_bc,
                            'activation': activation,
                            'rel_l2_error': rel_l2_error,
                            'max_error': max,
                            'u_pred': u_pred,
                            'u_exact': u_exact
                        })




df_pinn = pd.DataFrame(pinn_results)
df_pinn.to_csv('pinn_results.csv', index=False)



Training with lr=0.0001, depth=3, width=16, lambda_pde=0.1, lambda_ic_bc=1.0, activation=tanh


100%|██████████| 20/20 [00:00<00:00, 84.74it/s]


Training completed in 0.24 seconds
Training with lr=0.0001, depth=3, width=16, lambda_pde=0.1, lambda_ic_bc=1.0, activation=sine


100%|██████████| 20/20 [00:00<00:00, 80.12it/s]


Training completed in 0.25 seconds
Training with lr=0.0001, depth=3, width=16, lambda_pde=0.1, lambda_ic_bc=1.0, activation=swish


...

100%|██████████| 20/20 [00:00<00:00, 64.49it/s]


Training completed in 0.31 seconds
Training with lr=0.0001, depth=5, width=32, lambda_pde=1.0, lambda_ic_bc=1.0, activation=sine


100%|██████████| 20/20 [00:00<00:00, 60.57it/s]


Training completed in 0.33 seconds
Training with lr=0.0001, depth=5, width=32, lambda_pde=1.0, lambda_ic_bc=1.0, activation=swish


100%|██████████| 20/20 [00:00<00:00, 43.14it/s]

In [ ]:
# import meshio
# from AirPollution.crbe_fem import create_mesh, Domains, Models, MeshData
# # import numpy as np
# # import matplotlib.pyplot as plt
# # import matplotlib.tri as mtri
# # import os
# domain_size = 20.0
# Lx = Ly = domain_size  # Half-size of the domain
# T = 10.0  # End time
# D = 0.1  # Diffusion coefficient (small value leads to advection-dominated flow)
# vx, vy = 1.0, 0.5  # Velocity field
# sigma = 0.1

#  # Create mesh with 30 points per axis (higher resolution)
# mesh_file = create_mesh(128, domain_size=domain_size)
# mesh = meshio.read(mesh_file)

#  # Setup parameters
# domains = Domains(Lx=Lx, Ly=Ly, T=T)
# model_params = Models(vx=vx, vy=vy, D=D, sigma=sigma)
# n_steps = 256
# mesh_data = MeshData(mesh, domains, nt=n_steps)  # More time steps for accuracy


## **Training PINN model**

In [ ]:
# import meshio
# import AirPollution.pinn2 as pinn2
# import AirPollution.crbe_fem as crbe_fem
# from AirPollution.crbe_fem import create_mesh, Domains, Models, MeshData
# import numpy as np

# layers = [3] + [16] * 4 + [1]

# domain = [-20, 20, -20, 20]
# time_range = [0, 10]

# vx, vy = 1.0, 0.5
# v = [vx, vy]

# lambda_weights = {'pde': 1.0, 'ic': 10.0, 'bc': 10.0}
# learning_rate = 0.001
# epochs = 1000
# n_steps = 256
# domain_size = 20.0
# mesh_size = 128

# mesh_file = crbe_fem.create_mesh(mesh_size, domain_size=domain_size)
# mesh = meshio.read(mesh_file)
# mesh_data = crbe_fem.MeshData(mesh, domains, nt=n_steps)
# model_params = crbe_fem.Models(vx=vx, vy=vy, D=D, sigma=sigma)


# n_ic = round(0.2 * mesh_data.number_of_segments)
# n_bc = n_ic
# n_col = mesh_data.number_of_segments - n_ic - n_bc
# batch_sizes = {'pde': n_col, 'ic': n_ic, 'bc': n_ic}

# model = pinn2.PINN(layers, D, v)

# history = pinn2.train_pinn(model, domain, time_range, batch_sizes, learning_rate, epochs, lambda_weights)

In [ ]:
# print(batch_sizes)

In [ ]:
#@title Plot
import torch
T_max = 10.0
t_array = torch.linspace(0, T_max, n_steps)
xx, yy = mesh_data.midpoints[:,0], mesh_data.midpoints[:,1]

# pinn_solutions = np.zeros((n_steps, len(xx)))

# device = "cuda" if torch.cuda.is_available() else "cpu"

# for i in range(n_steps):
#     t_val = t_array[i]
#     t_tensor = torch.full((len(xx), 1), t_val, device=device)
#     # Explicitly cast x_tensor and y_tensor to torch.float32
#     x_tensor = torch.tensor(xx, device=device, dtype=torch.float32).reshape(-1, 1)
#     y_tensor = torch.tensor(yy, device=device, dtype=torch.float32).reshape(-1, 1)
#     # print(f"x_tensor shape: {x_tensor.shape}")
#     # print(f"y_tensor shape: {y_tensor.shape}")
#     xyt = torch.cat([x_tensor, y_tensor, t_tensor], dim=1)

#     with torch.no_grad():
#         pinn_solutions[i, :] = model(t_tensor, x_tensor, y_tensor).cpu().numpy().reshape(-1)

# # pinn_solutions.shape

# #Plotting

# import matplotlib.pyplot as plt
# import matplotlib.tri as mtri
# import os

# def plot_interpoleted_solution(time_index=None, save_dir="results"):
#         """
#         Plot solution at specified time index.

#         Parameters:
#         -----------
#         time_index: int or None
#             Time index to plot. If None, plot final time.
#         save_dir: str
#             Directory to save plots
#         """
#         if time_index is None:
#             time_index = mesh_data.nt - 1

#         t = 10 #* mesh_data.dt

#         # Create directory if it doesn't exist
#         os.makedirs(save_dir, exist_ok=True)

#         # Get points and triangles
#         points = mesh_data.points
#         triangles = mesh_data.triangles


#         # Get numerical solution at the specified time index
#         numerical_sol = pinn_solutions[-1,:]

#         # Map segment values to vertices for plotting
#         vertex_values = np.zeros(len(points))
#         count = np.zeros(len(points))

#         for i, (a, b) in enumerate(mesh_data.segments):
#             vertex_values[a] += numerical_sol[i]
#             vertex_values[b] += numerical_sol[i]
#             count[a] += 1
#             count[b] += 1

#         # Average values at vertices
#         vertex_values /= np.maximum(count, 1)  # Avoid division by zero

#         # Compute analytical solution at vertices
#         # t_tensor = torch.full((len(points), 1), t, device=device)
#         # xyt = torch.cat([self.mesh_data.points, t_tensor], dim=1)
#         # analytical_vertex_values = self.model_params.analytical_solution(xyt).cpu().numpy()
#         # Compute analytical solution at vertices
#         t_array = np.full((len(points), 1), t)
#         xyt = np.hstack([points, t_array])
#         analytical_vertex_values = model_params.analytical_solution(xyt)


#         # Error at vertices
#         error_values = vertex_values - analytical_vertex_values

#         # Create subplot
#         fig, axs = plt.subplots(1, 2, figsize=(15, 5))

#         # Plot numerical solution
#         triang = mtri.Triangulation(points[:, 0], points[:, 1], triangles)
#         cntr1 = axs[0].tricontourf(triang, vertex_values, 20, cmap="viridis")
#         axs[0].set_title(f"Numerical Solution at t = {t:.3f}")
#         axs[0].set_xlabel("x")
#         axs[0].set_ylabel("y")
#         fig.colorbar(cntr1, ax=axs[0])

#         # Plot analytical solution
#         cntr2 = axs[1].tricontourf(triang, analytical_vertex_values, 20, cmap="viridis")
#         axs[1].set_title(f"Analytical Solution at t = {t:.3f}")
#         axs[1].set_xlabel("x")
#         axs[1].set_ylabel("y")
#         fig.colorbar(cntr2, ax=axs[1])

#         # # Plot error
#         # cntr3 = axs[2].tricontourf(triang, error_values, 20, cmap="coolwarm",
#         #                            norm=plt.Normalize(-np.max(np.abs(error_values)),
#         #                                             np.max(np.abs(error_values))))
#         # axs[2].set_title(f"Error at t = {t:.3f}")
#         # axs[2].set_xlabel("x")
#         # axs[2].set_ylabel("y")
#         # fig.colorbar(cntr3, ax=axs[2])

#         plt.tight_layout()
#         plt.savefig(f"{save_dir}/pinn_solution_t{time_index}_interpolated.png", dpi=300)
#         plt.savefig(f"{save_dir}/pinn_solution_t{time_index}_interpolated.pdf", dpi=300)
#         plt.close()
#         print(f"Saved at {save_dir}/pinn_solution_t{time_index}_interpolated.png/pdf")



# plot_interpoleted_solution()

In [ ]:
# pinn2.plot_loss_history(history)

In [ ]:
# batch_sizes = {'pde': 1000, 'ic': 100, 'bc': 100}

# pinn2.plot_collocation_points(domain, time_range, batch_sizes)

In [ ]:
# mesh_file = crbe_fem.create_mesh(32, domain_size=domain_size)
# mesh = meshio.read(mesh_file)
# domains = Domains(Lx=Lx, Ly=Ly, T=T)
# mesh_data = crbe_fem.MeshData(mesh, domains, nt=n_steps)
# mesh_data.show()